In [1]:
!pip install image_classifiers
#!pip install keras==3.1.1 tensorflow==2.16.1
import tensorflow as tf
import keras
from keras.applications import ResNet50V2
from keras import Sequential, Input
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.datasets import cifar100
from keras.models import Model
from classification_models.keras import Classifiers
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import numpy as np

2024-04-28 10:58:10.188418: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-28 10:58:10.189413: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 10:58:10.196690: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 10:58:10.261023: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 10:58:11.368092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
batch_size = 100
num_classes = 100
n_images_test = 10000

In [3]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")
model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 134.58 MiB/s 1s

In [4]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [5]:
!mc cp s3/afeldmann/projet_cnam/model_etudiant_t1_a0.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_distille.keras
model_etudiant_distille = new_modele_resnet()
model_etudiant_distille.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_distille.keras")
model_etudiant_distille.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...1_a0.keras: 43.54 MiB / 43.54 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 106.12 MiB/s 0s

In [6]:
!mc cp s3/afeldmann/projet_cnam/model_etudiant_t1_a100.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_temoin.keras
model_etudiant_temoin = new_modele_resnet()
model_etudiant_temoin.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_temoin.keras")
model_etudiant_temoin.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...a100.keras: 43.54 MiB / 43.54 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 108.38 MiB/s 0s

In [7]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def load_cifar_test():
    _, test_dataset = cifar100.load_data()
    test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset).map(preprocessing, num_parallel_calls = tf.data.AUTOTUNE).batch(batch_size).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE)
    return test_dataset

In [8]:
@tf.function
def argmax_equals(x,y):
    return tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1))

In [9]:
test_dataset = load_cifar_test()
def make_burt(images, label):
    enseignant_pred = model_enseignant(images, training = False)
    etudiant_distille_pred = model_etudiant_distille(images, training = False)
    etudiant_temoin_pred = model_etudiant_temoin(images, training = False)
    bons_enseignant = argmax_equals(enseignant_pred,label)
    bons_etudiant_temoin = argmax_equals(etudiant_temoin_pred, label)
    bons_etudiant_distille = argmax_equals(etudiant_distille_pred, label)
    tdc = tf.cast(
        tf.stack([bons_enseignant,
                  tf.math.logical_not(bons_enseignant),
                  bons_etudiant_temoin,
                  tf.math.logical_not(bons_etudiant_temoin),
                  bons_etudiant_distille,
                  tf.math.logical_not(bons_etudiant_distille)],
                  axis=1),
        tf.int32)
    burt = tf.linalg.matmul(tf.transpose(tdc),tdc)
    return burt

burt = test_dataset.map(make_burt, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE).reduce(0, lambda x, y: x + y).numpy()

In [10]:
burt

array([[7651,    0, 6525, 1126, 6447, 1204],
       [   0, 2349,  748, 1601,  671, 1678],
       [6525,  748, 7273,    0, 6265, 1008],
       [1126, 1601,    0, 2727,  853, 1874],
       [6447,  671, 6265,  853, 7118,    0],
       [1204, 1678, 1008, 1874,    0, 2882]], dtype=int32)

In [11]:
burt_lignes = np.apply_along_axis(lambda x: np.round(x/x.sum()*300, 1), 1, burt)
burt_lignes

array([[100. ,   0. ,  85.3,  14.7,  84.3,  15.7],
       [  0. , 100. ,  31.8,  68.2,  28.6,  71.4],
       [ 89.7,  10.3, 100. ,   0. ,  86.1,  13.9],
       [ 41.3,  58.7,   0. , 100. ,  31.3,  68.7],
       [ 90.6,   9.4,  88. ,  12. , 100. ,   0. ],
       [ 41.8,  58.2,  35. ,  65. ,   0. , 100. ]])

In [12]:
np.savetxt("sauvegardes/burt.csv", burt, delimiter = ",")
np.savetxt("sauvegardes/burt_lignes.csv", burt_lignes, delimiter = ",")
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/burt.csv s3/afeldmann/projet_cnam/burt.csv
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/burt_lignes.csv s3/afeldmann/projet_cnam/burt_lignes.csv

...lignes.csv: 900 B / 900 B ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 7.75 KiB/s 0s]11;?\